pip install pyaudio

In [1]:
%load_ext autoreload
%autoreload 2

import Structures
import EmotionExtractor


em = EmotionExtractor.EmotionExtractor('baseline.npy', 'baseline_mean_sd.pickle')

Using TensorFlow backend.
Structures.py:91: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  bilstm = merge([lstm_fwd, lstm_bwd], name='bilstm', mode='concat')
/Users/julian/anaconda3/envs/Python27/lib/python2.7/site-packages/keras/legacy/layers.py:464: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
Structures.py:100: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  sent_representation = merge([attention,drop_out], dot_axes=1, mode='dot')
Structures.py:105: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=[<tf.Tenso...)`
  model = Model(input=[encoding_input], output=output)


In [2]:
from sys import byteorder
from array import array
from struct import pack
import time
import threading
from Queue import Queue
import numpy as np
import pyaudio
import wave
import os
from pydub import AudioSegment

In [3]:
THRESHOLD = 500
CHUNK_SIZE = 1024
FORMAT = pyaudio.paInt16
RATE = 8000
RECORD_SECONDS = 3
WAVE_OUTPUT_FILENAME_EXTENSION = 0
WAVE_OUTPUT_FILENAME = "output"

q = Queue()
lock = threading.Lock()

In [4]:
def is_silent(snd_data):
    "Returns 'True' if below the 'silent' threshold"
    return max(snd_data) < THRESHOLD

def normalize(snd_data):
    "Average the volume out"
    MAXIMUM = 16384
    times = float(MAXIMUM)/max(abs(i) for i in snd_data)

    r = array('h')
    for i in snd_data:
        r.append(int(i*times))
    return r

def trim(snd_data):
    "Trim the blank spots at the start and end"
    def _trim(snd_data):
        snd_started = False
        r = array('h')

        for i in snd_data:
            if not snd_started and abs(i)>THRESHOLD:
                snd_started = True
                r.append(i)

            elif snd_started:
                r.append(i)
        return r

    # Trim to the left
    snd_data = _trim(snd_data)

    # Trim to the right
    snd_data.reverse()
    snd_data = _trim(snd_data)
    snd_data.reverse()
    return snd_data

def add_silence(snd_data, seconds):
    "Add silence to the start and end of 'snd_data' of length 'seconds' (float)"
    r = array('h', [0 for i in range(int(seconds*RATE))])
    r.extend(snd_data)
    r.extend([0 for i in range(int(seconds*RATE))])
    return r

def record():
    """
    Record a word or words from the microphone and 
    return the data as an array of signed shorts.

    Normalizes the audio, trims silence from the 
    start and end.
    """
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=1, rate=RATE, input=True, output=True, frames_per_buffer=CHUNK_SIZE)

    num_silent = 0
    snd_started = False

    r = array('h')
    count = 0
    for i in range(0, int(RATE / CHUNK_SIZE * RECORD_SECONDS)):
        count +=1
        # little endian, signed short
        snd_data = array('h', stream.read(CHUNK_SIZE))
        if byteorder == 'big':
            snd_data.byteswap()
        r.extend(snd_data)

        #print('\r%08d' % count)
        #silent = is_silent(snd_data)

        #if silent and snd_started:
            #num_silent += 1
        #elif not silent and not snd_started:
            #snd_started = True

    sample_width = p.get_sample_size(FORMAT)
    stream.stop_stream()
    stream.close()
    p.terminate()

    r = normalize(r)
    #r = trim(r)
    #r = add_silence(r, 0.5)
    return sample_width, r

def record_to_file():
    with lock:
        WAVE_OUTPUT_FILENAME_EXTENSION = 0
        WAVE_OUTPUT_FILENAME = "rec"
        d= True
        
        ts = time.time()
        EXPORT_FOLDER = "RECORDINGS_" + str(ts).split(".")[0]
        if not os.path.exists(EXPORT_FOLDER):
            os.makedirs(EXPORT_FOLDER)
        
        # the sound of each 3 seconds intervall
        # they get combined later to a 9 seconds total interval
        first_recording = None
        second_recording = None
        third_recording = None
        while d:
            q.join()
            print('Recording')
            sample_width, data = record()
            # if there was no recording done so far
            # store the first 3 seconds in the first recording
            if first_recording == None:
                first_recording = data
            # if there was only one recording done so far
            # store the next recording in the second recording
            elif second_recording == None:
                second_recording = data
            # for every recording coming after the first 2 times 3 seconds
            # store the recording in the third recording
            else:
                third_recording = data
                    
                # append all the recordings to a 9 seconds interval
                data = first_recording
                data = np.append(data, second_recording)
                data = np.append(data, third_recording)
                # pack the data properly to be exported as a wave file
                data = pack('<' + ('h'*len(data)), *data)
                
                print('Saving........')
                wf = wave.open(EXPORT_FOLDER + "/" + WAVE_OUTPUT_FILENAME + "_" + str(WAVE_OUTPUT_FILENAME_EXTENSION) + ".wav", 'wb')
                wf.setnchannels(1)
                wf.setsampwidth(sample_width)
                wf.setframerate(RATE)
                wf.writeframes(data)
                wf.close()
                
                # 
                song = AudioSegment.from_wav(EXPORT_FOLDER + "/" + WAVE_OUTPUT_FILENAME + "_" + str(WAVE_OUTPUT_FILENAME_EXTENSION) + ".wav")
                song.export(EXPORT_FOLDER + "/" + WAVE_OUTPUT_FILENAME + "_" + str(WAVE_OUTPUT_FILENAME_EXTENSION) + ".mp3", format="mp3")
                song = AudioSegment.from_mp3(EXPORT_FOLDER + "/" + WAVE_OUTPUT_FILENAME + "_" + str(WAVE_OUTPUT_FILENAME_EXTENSION) + ".mp3")
                print("song")
                print(song.frame_rate)
                print(song.sample_width)
                result =  em.split_single_song(song)
#                 result["filename"] = i

                #dict_to_append = result.to_dict('record')
                
                #to_json.append(dict_to_append[0])
                #print result.to_dict('list')
                print("result")
                print result.to_dict('record')
                #print result_all.to_json()
                
                # increase the name counter of the filename
                WAVE_OUTPUT_FILENAME_EXTENSION += 1

                # shift the recordings and delete the last recording
                # shift the second recording to be the first now
                first_recording = None
                first_recording = second_recording
                # shift the third recording to be the second now
                second_recording = None
                second_recording = third_recording
                # empty the third recording so a new recording can be made
                third_recording = None
                
                d = False
            
            # Exit the loop with enter
            try:
                if q.get(timeout = 0) == 1:
                    d = False
            except:
                pass
        
        print('Task Completing')
        q.task_done()

In [ ]:
if __name__ == '__main__':
    print('Press Enter to stop Recording')
    t = threading.Thread(target=record_to_file)
    t.daemon=True
    t.start()
    input()
    q.put(1)
    q.join()
    print('Finished')

Press Enter to stop Recording
Recording
Recording
Recording
Saving........
song
8000
8000
8000
8000


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Users/julian/anaconda3/envs/Python27/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/Users/julian/anaconda3/envs/Python27/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "<ipython-input-4-af1297fc49a2>", line 142, in record_to_file
    result =  em.split_single_song(song)
  File "EmotionExtractor.py", line 126, in split_single_song
    prediction = self.my_attention_network.predict(np.array([convers]))[0]
  File "/Users/julian/anaconda3/envs/Python27/lib/python2.7/site-packages/keras/engine/training.py", line 1839, in predict
    self._make_predict_function()
  File "/Users/julian/anaconda3/envs/Python27/lib/python2.7/site-packages/keras/engine/training.py", line 1029, in _make_predict_function
    **kwargs)
  File "/Users/julian/anaconda3/envs/Python27/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py", line 2